# Day19_1: RAG (Retrieval-Augmented Generation) - 검색 증강 생성

## 📚 학습 목표

**Part 1: 기초**
1. LLM의 한계(Hallucination, 지식 단절) 이해하기
2. RAG 개념과 3단계 파이프라인 이해하기
3. 텍스트 임베딩과 벡터 개념 이해하기
4. 벡터 데이터베이스 개념 이해하기
5. 유사도 검색(Semantic Search) 이해하기

**Part 2: 심화**
1. 문서 청킹(Chunking) 전략 이해하기
2. LangChain으로 RAG 파이프라인 구축하기
3. Supabase 벡터 DB 연동하기
4. AI 뉴스 Q&A 봇 실습하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| RAG | 사내 문서 기반 AI 챗봇 | 고객센터 FAQ 봇, 사내 규정 검색 |
| 임베딩 | 의미 기반 검색 | 유사 문서 추천, 중복 콘텐츠 탐지 |
| 벡터 DB | 대규모 문서 검색 | 법률 문서 검색, 논문 검색 |
| LangChain | LLM 애플리케이션 개발 | 챗봇, 요약기, 질의응답 시스템 |

**분석가 관점**: GPT와 같은 LLM은 강력하지만, 최신 정보나 회사 내부 데이터에 접근할 수 없습니다. RAG는 이러한 한계를 극복하여 '우리 데이터로 답변하는 AI'를 만들 수 있게 해줍니다. 실무에서 가장 많이 활용되는 LLM 애플리케이션 패턴입니다!

---

# Part 1: 기초

---

## 1.1 LLM의 한계

### 대규모 언어 모델(LLM)의 놀라운 능력

GPT-4, Claude, Gemini 같은 LLM은 방대한 텍스트 데이터를 학습하여 인간과 유사한 언어를 구사합니다.
요약, 번역, 창작, 코딩 등 놀라운 작업을 수행하지만, 몇 가지 명백한 한계가 존재합니다.

### LLM의 4가지 주요 한계

| 한계 | 설명 | 예시 |
|------|------|------|
| **환각(Hallucination)** | 그럴듯한 거짓 정보 생성 | 존재하지 않는 논문 인용 |
| **지식 단절(Knowledge Cutoff)** | 학습 시점 이후 정보 부재 | "2025년 대통령은?" |
| **개인/도메인 정보 부재** | 내부 문서, 비공개 정보 접근 불가 | 사내 규정, 고객 DB |
| **출처 불명확** | 답변의 근거 제시 어려움 | "어디서 그 정보를 알았어?" |

In [ ]:
# LLM 한계 시연을 위한 예시
llm_limitations = {
    "환각 (Hallucination)": {
        "질문": "김철수 교수의 2024년 AI 윤리 논문에 대해 알려줘",
        "문제점": "LLM이 존재하지 않는 논문을 만들어내거나, 잘못된 인용을 생성할 수 있음"
    },
    "지식 단절 (Knowledge Cutoff)": {
        "질문": "어제 발표된 최신 AI 모델의 이름은?",
        "문제점": "LLM의 지식은 특정 시점에 고정되어 있어 최신 정보를 알 수 없음"
    },
    "개인/도메인 정보 부재": {
        "질문": "우리 회사의 휴가 정책이 뭐야?",
        "문제점": "내부 문서나 비공개 정보에 접근 불가"
    },
    "출처 불명확": {
        "질문": "AI가 일자리를 대체할 확률은?",
        "문제점": "답변의 근거나 출처를 명확히 제시하기 어려움"
    }
}

print("LLM의 주요 한계:")
print("=" * 70)
for limitation, details in llm_limitations.items():
    print(f"\n{limitation}")
    print(f"  예시 질문: {details['질문']}")
    print(f"  문제점: {details['문제점']}")

### 비유로 이해하기

```
LLM = 똑똑하지만 '오픈북'이 허용되지 않는 학생

- 암기력이 뛰어나지만, 시험 날 교재를 볼 수 없음
- 배운 내용은 잘 답하지만, 새로운 정보에는 추측으로 답함
- 자신감 있게 답하지만, 틀릴 수 있음

해결책 = RAG (오픈북을 쥐여주기!)
```

---

## 1.2 RAG란?

### RAG (Retrieval-Augmented Generation)

**검색 증강 생성** - LLM에게 '오픈북'을 쥐여주는 기술

```
전통적인 LLM:
  질문 -> LLM(기억만 의존) -> 답변

RAG:
  질문 -> 관련 문서 검색 -> LLM(문서 참고) -> 답변
```

### RAG의 3단계 파이프라인

| 단계 | 영어 | 설명 |
|------|------|------|
| **1. 인덱싱** | Indexing | 문서를 벡터로 변환하여 저장 (사전 준비) |
| **2. 검색** | Retrieval | 질문과 유사한 문서 조각 검색 |
| **3. 생성** | Generation | 검색된 문서를 바탕으로 답변 생성 |

In [ ]:
# RAG 파이프라인 구조 시각화
rag_pipeline = """
=== RAG 파이프라인 (3단계) ===

[1단계: 인덱싱 (Indexing) - 지식 창고 만들기]
  문서 로드 -> 청킹(분할) -> 임베딩(벡터화) -> 벡터 DB 저장
  
  예시:
  "2024년 매출은 100억입니다. 전년 대비 20% 증가했습니다."
  -> [청크1: "2024년 매출은 100억입니다.", 청크2: "전년 대비 20% 증가했습니다."]
  -> [벡터1: [0.1, 0.3, ...], 벡터2: [0.2, 0.4, ...]]
  -> 벡터 DB 저장

[2단계: 검색 (Retrieval) - 관련 정보 찾기]
  사용자 질문 -> 임베딩 -> 벡터 DB 검색 -> 유사 문서 반환
  
  예시:
  "올해 매출이 얼마야?" -> [0.1, 0.35, ...] -> 유사도 검색
  -> 청크1: "2024년 매출은 100억입니다." 반환 (유사도: 0.92)

[3단계: 생성 (Generation) - 검색된 정보로 답변하기]
  질문 + 검색된 문서 -> 프롬프트 구성 -> LLM -> 답변
  
  예시:
  프롬프트: "다음 Context를 바탕으로 답변하세요. Context: 2024년 매출은 100억입니다.
            질문: 올해 매출이 얼마야?"
  -> LLM: "2024년 매출은 100억입니다."
"""

print(rag_pipeline)

### 왜 RAG인가?

| 장점 | 설명 |
|------|------|
| **최신성** | 외부 데이터를 실시간으로 참조 |
| **정확성** | 사실에 기반한 답변 (환각 감소) |
| **확장성** | 내부 문서, 도메인 지식 쉽게 추가 |
| **투명성** | 답변의 출처 명확히 제시 가능 |
| **비용 효율** | 모델 재학습 없이 지식 업데이트 |

---

## 1.3 텍스트 임베딩

### 임베딩이란?

텍스트를 컴퓨터가 이해할 수 있는 **고차원 숫자 벡터**로 변환하는 기술

```
"안녕하세요" -> [0.12, -0.34, 0.56, ..., 0.78]  (768차원 또는 1536차원)

핵심 특징:
- 의미가 비슷한 텍스트 = 벡터 공간에서 가까이 위치
- "사과"와 "과일"은 가까움
- "사과"와 "자동차"는 멀리 위치
```

### 임베딩 모델 종류

| 모델 | 차원 | 특징 |
|------|------|------|
| OpenAI text-embedding-3-small | 1536 | 빠름, 저렴, 영어 최적화 |
| OpenAI text-embedding-3-large | 3072 | 고성능, 비용 높음 |
| sentence-transformers | 384~1024 | 오픈소스, 로컬 실행 |
| multilingual-e5 | 1024 | 다국어 지원, 한국어 우수 |

In [ ]:
# 필수 라이브러리 설치
# !pip install langchain langchain-openai langchain-community supabase python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 API 키 로드
load_dotenv()

# API 키 확인 (보안상 일부만 출력)
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"OpenAI API Key loaded: {api_key[:8]}...{api_key[-4:]}")
else:
    print("Warning: OPENAI_API_KEY not found!")
    print("Please set your API key in .env file or environment variable.")

In [ ]:
# OpenAI 임베딩 모델 사용
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 단일 텍스트 임베딩
text = "오늘 날씨가 좋습니다."
embedding = embedding_model.embed_query(text)

print(f"텍스트: {text}")
print(f"임베딩 차원: {len(embedding)}")
print(f"임베딩 벡터 (처음 10개): {embedding[:10]}")

In [ ]:
# 여러 텍스트의 의미적 유사도 비교
import numpy as np

def cosine_similarity(vec1, vec2):
    """두 벡터의 코사인 유사도 계산"""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 비교할 문장들
sentences = [
    "오늘 날씨가 좋습니다.",
    "날씨가 맑고 화창하네요.",
    "비가 와서 우산이 필요합니다.",
    "파이썬 프로그래밍을 배우고 있습니다."
]

# 임베딩 생성
embeddings = [embedding_model.embed_query(s) for s in sentences]

# 유사도 비교
print("문장 간 코사인 유사도:")
print("=" * 60)
for i in range(len(sentences)):
    for j in range(i+1, len(sentences)):
        sim = cosine_similarity(embeddings[i], embeddings[j])
        print(f"'{sentences[i][:15]}...' vs '{sentences[j][:15]}...': {sim:.4f}")

### 💡 실무 예시: 임베딩의 마법

```
의미가 비슷한 문장:
  "오늘 날씨가 좋습니다" ~ "날씨가 맑고 화창하네요" -> 유사도 높음 (0.8+)

의미가 다른 문장:
  "오늘 날씨가 좋습니다" ~ "파이썬 프로그래밍..." -> 유사도 낮음 (0.3-)
  
이것이 '의미 기반 검색(Semantic Search)'의 원리입니다!
```

---

## 1.4 벡터 데이터베이스

### 벡터 데이터베이스란?

임베딩 벡터를 효율적으로 저장하고, 유사한 벡터를 빠르게 검색하는 특수 데이터베이스

```
전통적인 DB:    SELECT * FROM docs WHERE content LIKE '%날씨%'
벡터 DB:        "비슷한 의미를 가진 문서 찾기" (코사인 유사도 기반)
```

### 주요 벡터 데이터베이스

| 벡터 DB | 특징 | 사용 사례 |
|---------|------|----------|
| **Chroma** | 로컬, 간단, 빠른 시작 | 프로토타입, 소규모 프로젝트 |
| **Supabase (pgvector)** | PostgreSQL 기반, 클라우드 | 프로덕션 서비스 |
| FAISS | 메타(Facebook), 고성능 | 대규모 검색, 추천 시스템 |
| Pinecone | 클라우드, 관리형 | 엔터프라이즈 |

In [ ]:
# Chroma 벡터 데이터베이스 사용하기 (로컬 테스트용)
from langchain_chroma import Chroma
from langchain.docstore.document import Document

# 샘플 문서 생성
documents = [
    Document(page_content="2024년 매출은 100억원입니다.", metadata={"source": "재무보고서"}),
    Document(page_content="전년 대비 매출이 20% 증가했습니다.", metadata={"source": "재무보고서"}),
    Document(page_content="직원 복지로 매년 휴가비를 지원합니다.", metadata={"source": "사내규정"}),
    Document(page_content="연차는 입사 1년 후부터 15일이 부여됩니다.", metadata={"source": "사내규정"}),
    Document(page_content="신제품 AI 스피커가 3월에 출시됩니다.", metadata={"source": "뉴스"}),
]

# 벡터 DB 생성 (in-memory)
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="demo_collection"
)

print(f"벡터 DB에 {len(documents)}개 문서 저장 완료!")

---

## 1.5 유사도 검색 (Similarity Search)

### 유사도 검색의 원리

```
1. 질문을 임베딩 벡터로 변환
2. 벡터 DB에서 가장 유사한 벡터들 찾기
3. 해당 문서(청크) 반환

유사도 측정 방법:
- 코사인 유사도 (Cosine Similarity): 가장 일반적
- 유클리드 거리 (Euclidean Distance)
- 내적 (Dot Product)
```

In [ ]:
# 유사도 검색 실행
query = "회사 매출이 얼마야?"
results = vectordb.similarity_search(query, k=3)

print(f"질문: {query}")
print("\n검색 결과:")
print("=" * 60)
for i, doc in enumerate(results, 1):
    print(f"\n[{i}] {doc.page_content}")
    print(f"    출처: {doc.metadata.get('source', 'N/A')}")

In [ ]:
# 유사도 점수와 함께 검색
results_with_scores = vectordb.similarity_search_with_score(query, k=3)

print(f"질문: {query}")
print("\n검색 결과 (유사도 점수 포함):")
print("=" * 60)
for i, (doc, score) in enumerate(results_with_scores, 1):
    print(f"\n[{i}] {doc.page_content}")
    print(f"    출처: {doc.metadata.get('source', 'N/A')}")
    print(f"    거리 점수: {score:.4f} (낮을수록 유사)")

In [ ]:
# 다양한 질문으로 테스트
test_queries = [
    "휴가 정책이 뭐야?",
    "새로 출시되는 제품은?",
    "작년보다 실적이 좋아졌어?"
]

print("다양한 질문에 대한 검색 결과:")
print("=" * 70)

for query in test_queries:
    results = vectordb.similarity_search(query, k=2)
    print(f"\n질문: {query}")
    for i, doc in enumerate(results, 1):
        print(f"  [{i}] {doc.page_content}")

---

# Part 2: 심화

---

## 2.1 문서 청킹 (Chunking) 전략

### 청킹이란?

긴 문서를 작은 조각(chunk)으로 나누는 과정

```
왜 청킹이 필요한가?
1. LLM 토큰 제한: GPT-4는 한 번에 128K 토큰만 처리 가능
2. 정확한 검색: 긴 문서보다 관련 부분만 검색하는 것이 효과적
3. 비용 효율: 필요한 부분만 LLM에 전달
```

### 청킹 전략

| 전략 | 설명 | 적합한 경우 |
|------|------|------------|
| Fixed Size | 고정 글자 수로 분할 | 간단한 문서 |
| Recursive Character | 문단/문장 경계 고려 분할 | 일반 텍스트 |
| Semantic | 의미 단위로 분할 | 복잡한 문서 |

### 중요한 파라미터

| 파라미터 | 설명 | 권장 값 |
|---------|------|--------|
| chunk_size | 각 청크의 최대 크기 | 500~1000 |
| chunk_overlap | 청크 간 중복 문자 수 | 50~100 |

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 샘플 긴 문서
long_document = """
AI 기술의 발전은 우리 사회에 많은 변화를 가져오고 있습니다.

첫째, 의료 분야에서 AI는 질병 진단의 정확도를 크게 향상시켰습니다. 
특히 영상 의학에서 AI가 종양을 발견하는 능력은 전문의 수준에 도달했습니다.
또한 신약 개발 기간을 획기적으로 단축시키고 있습니다.

둘째, 교육 분야에서 AI 튜터가 개인화된 학습을 제공합니다.
학생 개개인의 수준에 맞춰 학습 내용을 조정하고,
취약한 부분을 집중적으로 보완해줍니다.

셋째, 금융 분야에서 AI는 사기 탐지와 리스크 관리에 활용됩니다.
실시간으로 수백만 건의 거래를 분석하여 이상 패턴을 감지합니다.
"""

# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,       # 각 청크의 최대 크기
    chunk_overlap=50,     # 청크 간 중복 글자 수
    separators=["\n\n", "\n", " ", ""]  # 분할 우선순위
)

# 문서 분할
chunks = text_splitter.split_text(long_document)

print(f"원본 문서 길이: {len(long_document)} 글자")
print(f"청크 수: {len(chunks)}개")
print("\n각 청크 내용:")
print("=" * 60)
for i, chunk in enumerate(chunks, 1):
    print(f"\n[청크 {i}] ({len(chunk)}글자)")
    print(f"{chunk[:100]}..." if len(chunk) > 100 else chunk)

---

## 2.2 LangChain으로 RAG 구현하기

### LangChain 소개

LLM 애플리케이션 개발을 위한 프레임워크

```
주요 구성 요소:
1. LLM: 언어 모델 (OpenAI, Anthropic 등)
2. Prompts: 프롬프트 템플릿
3. Chains: 여러 컴포넌트 연결
4. Retrievers: 문서 검색
5. Memory: 대화 기록 관리
```

### RAG 체인 구성

```
리트리버(Retriever) -> 문서 결합 체인 -> LLM -> 답변
```

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 리트리버 생성 (벡터 DB에서 검색)
retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # 상위 3개 문서 반환
)

print("LLM과 Retriever 설정 완료!")
print(f"LLM: {llm.model_name}")
print(f"검색 결과 수: {retriever.search_kwargs['k']}개")

---

## 2.3 프롬프트 템플릿 활용하기

### 프롬프트 템플릿의 중요성

```
좋은 프롬프트 = 좋은 답변

핵심 요소:
1. 역할 부여: "당신은 ~ 전문가입니다"
2. Context 제공: 검색된 문서 삽입 위치
3. 지시사항: 답변 규칙 명시
4. 질문 삽입: 사용자 질문 위치
```

In [ ]:
# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template("""
당신은 친절하고 정확한 AI 어시스턴트입니다.
주어진 'Context'를 바탕으로 질문에 답변해주세요.

규칙:
1. Context에 있는 정보만 사용하세요.
2. Context에 답이 없으면 "주어진 정보에서 답을 찾을 수 없습니다."라고 말하세요.
3. 추측하거나 만들어내지 마세요.
4. 답변 후 출처(source)를 명시하세요.

[Context]
{context}

[질문]
{input}

[답변]
""")

print("프롬프트 템플릿 예시:")
print(prompt.format(context="예시 문서 내용", input="예시 질문"))

In [ ]:
# RAG 체인 생성
document_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, document_chain)

print("RAG 체인 생성 완료!")

In [ ]:
# RAG 체인 테스트
question = "회사 매출이 얼마야?"
response = rag_chain.invoke({"input": question})

print(f"질문: {question}")
print(f"\n답변: {response['answer']}")
print(f"\n사용된 문서:")
for i, doc in enumerate(response['context'], 1):
    print(f"  [{i}] {doc.page_content}")

In [ ]:
# 다양한 질문 테스트
questions = [
    "휴가는 며칠이나 받을 수 있어?",
    "신제품은 언제 출시돼?",
    "회사 주소가 어디야?"  # Context에 없는 질문
]

print("RAG 시스템 테스트:")
print("=" * 70)

for q in questions:
    response = rag_chain.invoke({"input": q})
    print(f"\n질문: {q}")
    print(f"답변: {response['answer']}")

---

## 2.4 Supabase 벡터 DB 연동하기

### Supabase 소개

Supabase는 오픈소스 PostgreSQL 기반의 백엔드 서비스 플랫폼으로, `pgvector` 확장을 통해 강력한 벡터 데이터베이스 기능을 제공합니다.

| 특징 | 설명 |
|------|------|
| PostgreSQL 기반 | 안정성과 확장성 |
| pgvector | 벡터 검색 지원 |
| 클라우드 호스팅 | 관리 편의성 |
| 무료 티어 | 프로토타입에 적합 |

In [ ]:
# Supabase 연결 테스트
from supabase import create_client

supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_ANON_KEY")

if supabase_url and supabase_key:
    print(f"Supabase URL: {supabase_url}")
    print(f"Key exists: {bool(supabase_key)}")
    
    # 기본 연결 테스트
    try:
        supabase = create_client(supabase_url, supabase_key)
        print("Supabase 연결 성공!")
    except Exception as e:
        print(f"Supabase 연결 실패: {e}")
else:
    print("Warning: Supabase 환경변수가 설정되지 않았습니다.")
    print("Chroma DB를 대신 사용합니다.")

---

## 2.5 AI 뉴스 Q&A 봇 실습

### 실습 시나리오

최신 AI 뉴스 기사를 기반으로 질문에 답변하는 Q&A 봇 만들기

```
전체 프로세스:
1. 뉴스 기사 준비
2. 청킹 및 임베딩
3. 벡터 DB 저장
4. RAG 체인 구성
5. Q&A 실행
```

In [ ]:
# 샘플 AI 뉴스 기사
news_articles = [
    {
        "title": "AI, 신약 개발 속도 10배 앞당긴다",
        "content": """
        2025년, 인공지능(AI)이 신약 개발 분야에서 혁신을 주도하고 있다. 
        전통적으로 10년 이상 소요되던 신약 개발 기간이 AI 기반 시뮬레이션과 데이터 분석을 통해 평균 1~2년으로 단축될 전망이다. 
        글로벌 제약사 '뉴로젠'은 최근 AI 플랫폼 '제네시스-1'을 활용해 알츠하이머 치료제 후보 물질을 6개월 만에 발굴했다고 발표했다. 
        이 플랫폼은 수백만 개의 화합물 구조를 가상으로 생성하고, 단백질과의 결합 가능성을 예측하여 유효 물질을 빠르게 선별한다. 
        전문가들은 AI가 개인 맞춤형 치료제 개발을 가속화하고, 희귀병 정복의 새로운 희망이 될 것이라고 평가했다.
        """,
        "source": "AI 타임즈",
        "date": "2025-01-15"
    },
    {
        "title": "데이터브릭스, 클라우드 비용 최적화 AI 스타트업 '옵티마이즈' 인수",
        "content": """
        데이터 및 AI 기업 데이터브릭스가 클라우드 비용 관리 AI 스타트업 '옵티마이즈(Optimize.AI)'를 5억 달러에 인수한다고 발표했다. 
        옵티마이즈는 AI를 사용해 기업의 클라우드 사용 패턴을 실시간으로 분석하고, 불필요한 자원을 자동으로 축소하거나 저렴한 플랜으로 전환하여 비용을 최대 40%까지 절감하는 솔루션을 제공한다. 
        데이터브릭스는 이번 인수를 통해 자사의 데이터 인텔리전스 플랫폼에 강력한 비용 최적화 기능을 통합, 
        기업들이 데이터 분석 및 AI 모델 운영 비용을 획기적으로 줄일 수 있도록 지원할 계획이라고 밝혔다. 
        이는 생성형 AI 도입이 확산되면서 급증하는 기업들의 클라우드 비용 부담을 해결하기 위한 전략적 행보다.
        """,
        "source": "테크 위클리",
        "date": "2025-01-18"
    },
    {
        "title": "온디바이스 AI, 2025년 스마트폰의 새로운 표준으로",
        "content": """
        클라우드를 거치지 않고 스마트폰, 노트북 등 기기 자체에서 AI 연산을 수행하는 '온디바이스 AI(On-device AI)'가 2025년 기술 업계의 최대 화두로 떠올랐다. 
        온디바이스 AI는 빠른 응답 속도, 강화된 개인정보 보호, 인터넷 연결 없는 AI 기능 사용 등의 장점을 가진다. 
        글로벌 스마트폰 제조사 '테크노바'는 차세대 플래그십 모델 'T-25'에 자체 개발한 신경망처리장치(NPU) '뉴로엔진 X'를 탑재, 
        실시간 통역, AI 사진 편집, 스마트 비서 기능을 오프라인에서도 완벽하게 지원한다고 밝혔다. 
        애플과 구글 또한 차세대 모바일 칩에 더욱 강력한 AI 처리 성능을 예고하고 있어, 온디바이스 AI 시장의 주도권 경쟁은 더욱 치열해질 전망이다.
        """,
        "source": "모바일 월드",
        "date": "2025-01-20"
    }
]

print(f"준비된 뉴스 기사: {len(news_articles)}개")
for article in news_articles:
    print(f"  - {article['title']} ({article['source']})")

In [ ]:
# Document 객체로 변환
news_documents = [
    Document(
        page_content=article["content"].strip(),
        metadata={
            "title": article["title"],
            "source": article["source"],
            "date": article["date"]
        }
    )
    for article in news_articles
]

print(f"총 {len(news_documents)}개의 Document 생성")

In [ ]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

split_docs = text_splitter.split_documents(news_documents)

print(f"분할 후 청크 수: {len(split_docs)}개")
for i, doc in enumerate(split_docs, 1):
    print(f"  [{i}] {doc.metadata['title'][:20]}... ({len(doc.page_content)}글자)")

In [ ]:
# 새로운 벡터 DB 생성 (뉴스 전용)
news_vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="news_collection"
)

print("뉴스 벡터 DB 생성 완료!")

In [ ]:
# 뉴스 Q&A 봇용 프롬프트 템플릿
news_prompt = ChatPromptTemplate.from_template("""
당신은 'AI 뉴스 전문 앵커'입니다. 
최신 AI 뉴스에 대한 질문에 친절하고 명확하게 답변해주세요.

규칙:
1. 주어진 Context에 있는 정보만 사용하세요.
2. 답변은 간결하고 핵심적으로 작성하세요.
3. 가능하면 구체적인 숫자나 데이터를 포함하세요.
4. 답변 끝에 출처(source)와 날짜를 명시하세요.
5. Context에 답이 없으면 "해당 정보가 뉴스에 없습니다."라고 말하세요.

[Context]
{context}

[질문]
{input}

[답변]
""")

# 뉴스 RAG 체인 생성
news_retriever = news_vectordb.as_retriever(search_kwargs={"k": 3})
news_document_chain = create_stuff_documents_chain(llm, news_prompt)
news_rag_chain = create_retrieval_chain(news_retriever, news_document_chain)

print("뉴스 Q&A 봇 준비 완료!")

In [ ]:
# 뉴스 Q&A 봇 테스트
news_questions = [
    "데이터브릭스가 인수한 회사는 어디야?",
    "신약 개발에 AI가 어떻게 사용돼?",
    "온디바이스 AI의 장점은 뭐야?",
    "뉴로젠은 어떤 AI 플랫폼을 사용해?",
    "삼성전자의 AI 전략은?"  # Context에 없는 질문
]

print("AI 뉴스 Q&A 봇 테스트:")
print("=" * 70)

for q in news_questions:
    response = news_rag_chain.invoke({"input": q})
    print(f"\n질문: {q}")
    print(f"답변: {response['answer']}")
    print("-" * 70)

In [ ]:
# 대화형 Q&A 함수
def ask_news_bot(question):
    """뉴스 Q&A 봇에게 질문하기"""
    response = news_rag_chain.invoke({"input": question})
    
    print(f"\n질문: {question}")
    print(f"\n답변: {response['answer']}")
    
    print(f"\n참조한 문서:")
    for i, doc in enumerate(response['context'], 1):
        print(f"  [{i}] {doc.metadata.get('title', 'N/A')} ({doc.metadata.get('source', 'N/A')})")
    
    return response

# 테스트
ask_news_bot("AI가 의료 분야에서 어떻게 활용되고 있어?")

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. LLM 한계 이해하기 ⭐

**문제**: LLM의 4가지 주요 한계(환각, 지식 단절, 개인정보 부재, 출처 불명확) 중 RAG로 해결할 수 있는 것들을 설명하고, 각각 어떻게 해결되는지 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q2. RAG 파이프라인 이해하기 ⭐

**문제**: RAG의 3단계 파이프라인(인덱싱, 검색, 생성)을 각각 한 문장으로 설명하고, 각 단계에서 어떤 도구가 사용되는지 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q3. 임베딩 직접 사용하기 ⭐⭐

**문제**: 아래 세 문장의 임베딩을 생성하고, 코사인 유사도를 계산하세요. 가장 유사한 문장 쌍은 무엇인가요?

```python
sentences = [
    "머신러닝은 데이터로부터 패턴을 학습합니다.",
    "딥러닝은 인공신경망을 활용한 기계학습입니다.",
    "오늘 점심은 김치찌개를 먹었습니다."
]
```

In [ ]:
sentences = [
    "머신러닝은 데이터로부터 패턴을 학습합니다.",
    "딥러닝은 인공신경망을 활용한 기계학습입니다.",
    "오늘 점심은 김치찌개를 먹었습니다."
]

# 여기에 코드를 작성하세요


### Q4. 벡터 DB 검색하기 ⭐⭐

**문제**: 본문에서 생성한 `news_vectordb`를 사용하여 "클라우드 비용"에 관한 문서를 검색하고, 결과를 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 유사도 검색 이해하기 ⭐⭐

**문제**: `similarity_search`와 `similarity_search_with_score`의 차이점을 설명하고, 유사도 점수(score)가 의미하는 바를 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q6. 청킹 전략 이해하기 ⭐⭐⭐

**문제**: `chunk_size`와 `chunk_overlap`의 역할을 설명하고, 각각을 너무 크게/작게 설정했을 때의 문제점을 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q7. 프롬프트 템플릿 수정하기 ⭐⭐⭐

**문제**: 본문의 뉴스 Q&A 봇 프롬프트를 수정하여, 답변 시 반드시 "핵심 요약", "상세 설명", "출처" 세 부분으로 구성되도록 만드세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q8. RAG 체인 구현하기 ⭐⭐⭐⭐

**문제**: 아래 문서들을 사용하여 새로운 RAG 체인을 구현하고, 질문에 답변하세요.

```python
documents = [
    "파이썬은 1991년 귀도 반 로섬이 개발한 프로그래밍 언어입니다.",
    "파이썬은 가독성이 좋고 배우기 쉬워 초보자에게 추천됩니다.",
    "데이터 과학과 AI 분야에서 파이썬이 가장 많이 사용됩니다.",
    "자바스크립트는 웹 개발의 표준 언어입니다."
]

질문: "파이썬은 누가 만들었고, 어떤 분야에서 많이 사용돼?"
```

In [ ]:
documents = [
    "파이썬은 1991년 귀도 반 로섬이 개발한 프로그래밍 언어입니다.",
    "파이썬은 가독성이 좋고 배우기 쉬워 초보자에게 추천됩니다.",
    "데이터 과학과 AI 분야에서 파이썬이 가장 많이 사용됩니다.",
    "자바스크립트는 웹 개발의 표준 언어입니다."
]

# 여기에 코드를 작성하세요


### Q9. RAG 시스템 분석하기 ⭐⭐⭐⭐

**문제**: RAG 시스템에서 검색 결과가 좋지 않을 때 (관련 없는 문서가 검색될 때) 개선할 수 있는 방법 3가지를 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q10. 나만의 Q&A 봇 만들기 ⭐⭐⭐⭐⭐

**문제**: 아래 주제 중 하나를 선택하여 최소 5개의 문서를 작성하고, Q&A 봇을 구현하세요. 최소 3개의 질문으로 테스트하세요.

주제 선택:
1. 회사 FAQ (휴가, 복지, 규정 등)
2. 제품 매뉴얼
3. 관심 분야 지식 베이스

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|------|----------|----------|
| LLM 한계 | 환각, 지식 단절, 내부정보 부재 | RAG로 극복 |
| RAG | 검색 + 생성 조합 | 사내 문서 기반 AI |
| 임베딩 | 텍스트 -> 벡터 변환 | 의미 기반 검색 |
| 벡터 DB | 벡터 저장 및 검색 | Chroma, Supabase |
| 유사도 검색 | 코사인 유사도 기반 | Top-K 문서 반환 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 내용 | 권장 설정 |
|------|----------|----------|
| 청킹 | 긴 문서 분할 | chunk_size: 500-1000 |
| 오버랩 | 문맥 연결 유지 | chunk_overlap: 50-100 |
| LangChain | RAG 파이프라인 구축 | 체인 패턴 활용 |
| 프롬프트 | Context + 지시사항 | 명확한 규칙 제시 |

### RAG 구현 체크리스트

```
1. 데이터 준비:
   [ ] 문서 수집 및 정제
   [ ] 청킹 전략 결정 (크기, 오버랩)
   
2. 인덱싱:
   [ ] 임베딩 모델 선택
   [ ] 벡터 DB 설정 (Chroma, Supabase 등)
   [ ] 문서 임베딩 및 저장
   
3. 검색:
   [ ] Retriever 설정 (k값 결정)
   [ ] 검색 품질 테스트
   
4. 생성:
   [ ] LLM 선택 (GPT-4, Claude 등)
   [ ] 프롬프트 템플릿 설계
   [ ] RAG 체인 구성
   
5. 평가:
   [ ] 다양한 질문 테스트
   [ ] 답변 품질 검증
   [ ] 출처 정확성 확인
```

### 💡 실무 팁

1. **임베딩 모델 선택**: 한국어 -> multilingual 모델 권장
2. **청크 크기**: 너무 크면 노이즈, 너무 작으면 문맥 손실
3. **k 값 설정**: 3-5개가 일반적, 많으면 비용 증가
4. **프롬프트 설계**: "Context에 없으면 모른다고 답변" 규칙 필수
5. **메타데이터 활용**: 출처, 날짜 정보로 답변 신뢰성 향상